In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

In [ ]:
df=pd.read_csv("data.csv",sep=";")
df=df.drop(0)
df.drop('Unnamed: 0', axis=1, inplace=True)
df=df.drop('Y',axis=1)
df_labels=df.Y
scaler = MinMaxScaler()
scaler.fit(df)
df=scaler.transform(df_train)
X_train, X_test, y_train, y_test = train_test_split(df,df_labels, test_size=0.30, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE (random_state=42)
X_res, Y_res = sm.fit_resample(X_train, y_train)
print (X_res.shape, Y_res.shape)

In [ ]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Dense(22, activation="relu"),
      layers.Dense(15, activation="relu"),
      layers.Dense(8, activation="relu")])
    
    self.decoder = tf.keras.Sequential([
      layers.Dense(15, activation="relu"),
      layers.Dense(22, activation="relu"),
      layers.Dense(23, activation="sigmoid")])
    
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

In [ ]:
autoencoder.compile(optimizer='adam', loss='mae')
history = autoencoder.fit(X_res,X_res, 
          epochs=20, 
          batch_size=512,
          shuffle=True)

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.legend()